In [6]:
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Praveen9375#',
    db='sqlalchemy'
)

cur = conn.cursor()
cur.execute("SELECT 1")
result = cur.fetchone()
print(result)

cur.close()
conn.close()

(1,)


In [ ]:
from flask import Flask, request, jsonify
from flask_restful import Api, Resource
from flask_mysqldb import MySQL

app = Flask(__name__)
api = Api(app)

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'Sagarmatha@123'
app.config['MYSQL_DB'] = 'new_schema'

mysql = MySQL(app)

class BaseResource(Resource):
    def get(self, report_type=None):
        return {'message': 'GET method call for database'}

    def post(self, report_type=None):
        sdate = request.json['sdate']
        edate = request.json['edate']
        accountid = request.json['accountid']
        return {'message': f'POST method call with sdate: {sdate}, edate: {edate}, accountid: {accountid}'}

class HomeResource(BaseResource):
    def __init__(self):
        super().__init__()
        self.methods = {
            'glance': self.handle_report,
            'daywise': self.handle_report
        }

    def get(self, report_type=None):
        if report_type in self.methods:
            return self.methods[report_type]()
        return super().get(report_type)

    def handle_report(self):
        cur = mysql.connection.cursor()
        sdate = request.json.get('sdate')
        edate = request.json.get('edate')
        accountid = request.json.get('accountid')

        if request.path == '/home/glance':
            query = f"SELECT SUM(spend) AS totalspend, accountid FROM fact_table WHERE fdate BETWEEN '{sdate}' AND '{edate}' AND accountid = {accountid} GROUP BY accountid"
        elif request.path == '/home/daywise':
            query = f"SELECT SUM(spend) AS totalspend, accountid, DAY(fdate) AS day FROM fact_table WHERE fdate BETWEEN '{sdate}' AND '{edate}' AND accountid = {accountid} GROUP BY accountid, day"

        cur.execute(query)
        data = cur.fetchall()
        cur.close()
#for return result in json format
        result = []
        for row in data:
            result.append({
                'totalspend': row[0],
                'accountid': row[1],
                'day': row[2] if len(row) > 2 else None
            })

        return jsonify(result)


resources = [
    (['/home', '/home/', '/home/<string:report_type>'], HomeResource)
]
# Dynamically handle resources
for routes, resource_class in resources:
    api.add_resource(resource_class, *routes)

if _name_ == '__main__':
    app.run(debug=True)

In [5]:
from flask import Flask
import pymysql

In [6]:
app = Flask(__name__)
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'Praveen9375#'
app.config['MYSQL_DB'] = 'sqlalchemy'

In [7]:

def get_db():
    conn = pymysql.connect(
        host=app.config['MYSQL_HOST'],
        user=app.config['MYSQL_USER'],
        password=app.config['MYSQL_PASSWORD'],
        db=app.config['MYSQL_DB']
    )
    return conn

In [8]:
@app.route('/')
def index():
    conn = get_db()
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS New_Table (id INT PRIMARY KEY AUTO_INCREMENT, name VARCHAR(255), age INT, department VARCHAR(255))")
    conn.commit()
    cur.close()
    conn.close()
    return "Table created successfully!"

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2023 07:41:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2023 07:41:43] "GET /favicon.ico HTTP/1.1" 404 -
